# 영화 평점 분석 실습

실습관련 YouTube 영상  
https://youtu.be/krmthaX9WD4  
https://youtu.be/bbSDVNYUmb4  
https://youtu.be/JOska9sZVrw

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## 1. 영화 평점 데이터 적재 및 전처리

In [10]:
# 사용자 데이터 읽어오기
users = pd.read_csv('data/movielens/users.dat', 
                    sep = '::', 
                    engine = 'python', 
                    names = ['사용자아이디', '성별','연령','직업','지역'])
users.head()

,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [11]:
# 평점 데이터 읽어오기
ratings = pd.read_csv('data/movielens/ratings.dat',
                      sep = '::', 
                      engine = 'python',
                      names = ['사용자아이디', '영화아이디','평점','타임스탬프'])
ratings.head()

,사용자아이디,영화아이디,평점,타임스탬프
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [12]:
# 영화데이터 읽어오기
movies = pd.read_csv('data/movielens/movies.dat',
                     sep = '::', 
                     engine = 'python',
                     names = ['영화아이디','영화제목','장르'], 
                     encoding = 'latin-1')
movies.head()

,영화아이디,영화제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
#3개의 데이터프레임을 하나로 합치기
data = pd.merge(users,ratings,on='사용자아이디')
data.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프
0,1,F,1,10,48067,1193,5,978300760
1,1,F,1,10,48067,661,3,978302109
2,1,F,1,10,48067,914,3,978301968
3,1,F,1,10,48067,3408,4,978300275
4,1,F,1,10,48067,2355,5,978824291


In [14]:
data = pd.merge(data,movies,on='영화아이디')
data.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


## 2. 보고 싶은 영화  찾기 (평점 높은 영화 찾기)

In [23]:
data.영화제목.unique().size

3706

In [24]:
data.영화아이디.unique().size # 영화 제목이 안 겹친다는걸 알 수 있음

3706

In [26]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 
data.pivot_table(index='영화제목', # 영화제목이 안 겹치니까
                 aggfunc='mean',
                 values='평점').sort_values(by='평점',ascending=False).head(10)

,평점
영화제목,
Ulysses (Ulisse) (1954),5.0
Lured (1947),5.0
Follow the Bitch (1998),5.0
Bittersweet Motel (2000),5.0
Song of Freedom (1936),5.0
One Little Indian (1973),5.0
Smashing Time (1967),5.0
Schlafes Bruder (Brother of Sleep) (1995),5.0
"Gate of Heavenly Peace, The (1995)",5.0


In [27]:
# 영화들의 평점 갯수도 함께 구하기
영화평점 = data.pivot_table(index='영화제목',aggfunc=['mean','count'],values=['평점'])
영화평점.columns = ['평균','갯수']

In [28]:
영화평점.head()

,평균,갯수
영화제목,,
"$1,000,000 Duck (1971)",3.027027,37
'Night Mother (1986),3.371429,70
'Til There Was You (1997),2.692308,52
"'burbs, The (1989)",2.910891,303
...And Justice for All (1979),3.713568,199


In [31]:
# 평점의 갯수가 2000개 이상이고 평균이 4.5이상인 영화를 평균으로 내림차순
영화평점[(영화평점.갯수>=2000) & (영화평점.평균 > 4.3)].sort_values(by='평균',ascending=False)

,평균,갯수
영화제목,,
"Shawshank Redemption, The (1994)",4.554558,2227
"Godfather, The (1972)",4.524966,2223
Schindler's List (1993),4.510417,2304
Raiders of the Lost Ark (1981),4.477725,2514
Star Wars: Episode IV - A New Hope (1977),4.453694,2991
"Sixth Sense, The (1999)",4.406263,2459
"Silence of the Lambs, The (1991)",4.351823,2578
Saving Private Ryan (1998),4.337354,2653
American Beauty (1999),4.317386,3428


## 3. (실습) 여자들이 좋아하는 영화 찾기 (평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화)

### 3-1. (심화) 여자들이 어떤 장르의 영화를 좋아하는지 찾아보자.(위에서 찾은 여성들이 좋아하는 영화들의 장르를 찾아낸 후, 어떤 장르가 많았는지 확인)

## (실습) 4. 남자와 여자의 호불호가 크게 갈리는 영화 10개를 출력 
### 전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

## 5. (실습) 연령대 별로 영화 평점 분석하기
### 연령대(10대 미만, 10대, 20대, ...50대) 컬럼을 추가한 후, 연령대로 집계를 수행하여 영화별 연령대별 영화평점 구하기